In [10]:
import pandas as pd
import pickle
from collections import Counter
import copy

import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier




In [11]:

pd.set_option('display.max_rows', 300)

survey = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/Classification-Project-DSjobs/Data/kaggle-survey-2017/multi_r_.csv', encoding = 'ISO-8859-1')

list(survey.columns)

survey.JobSatisfaction.value_counts()



survey.head()

survey.EmploymentStatus.unique()

survey.groupby('EmploymentStatus').count()

# survey.UniversityImportance.value_counts()

# survey.JobSatisfaction.value_counts()




/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (32,84,87,88,100,110,117,124,125,128,130,131,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,GenderSelect,Country,Age,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorExperienceLevel,JobFactorDepartment,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity
EmploymentStatus,,,,,,,,,,,,,,,,,,,,,
Employed full-time,10897,10853,10840,10702,0,0,10843,2400,9787,9533,...,1066,1060,1061,1062,1062,1068,1065,1063,1065,1075
Employed part-time,917,910,907,901,0,0,903,288,787,751,...,132,131,132,131,132,133,132,134,133,132
I prefer not to say,420,404,399,382,380,383,0,0,0,0,...,190,188,189,187,187,188,188,187,187,187
"Independent contractor, freelancer, or self-employed",1330,1321,1321,1302,0,0,1322,281,1176,1142,...,105,104,104,104,104,106,106,103,104,105
"Not employed, and not looking for work",924,919,915,909,900,901,0,0,0,0,...,522,522,519,521,519,525,520,521,524,520
"Not employed, but looking for work",2110,2098,2096,2075,0,0,0,0,0,0,...,1422,1411,1409,1406,1390,1404,1398,1393,1397,1405
Retired,118,116,117,114,0,0,118,43,80,78,...,0,0,0,0,0,0,0,0,0,0


In [12]:
conditions = [  survey.JobSatisfaction.isin(['7','8','9','10 - Highly Satisfied']),
                survey.JobSatisfaction.isin(['1 - Highly Dissatisfied','2','3','4','5','6','I prefer not to share'])
             ]
choices = [1,0]
survey['JobSatisfaction'] = np.select(conditions, choices, default=survey.JobSatisfaction)


survey = survey[survey.JobSatisfaction.notna()]

In [13]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
missing_value_df

,columnname,percent_missing
Unnamed: 0,Unnamed: 0,0.000000
GenderSelect,GenderSelect,0.194698
Country,Country,0.284559
Age,Age,1.258050
EmploymentStatus,EmploymentStatus,0.000000
StudentStatus,StudentStatus,100.000000
LearningDataScience,LearningDataScience,100.000000
CodeWriter,CodeWriter,0.000000
CareerSwitcher,CareerSwitcher,100.000000
CurrentJobTitleSelect,CurrentJobTitleSelect,0.029954


In [14]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
seventy_five_data = missing_value_df[missing_value_df.percent_missing<25]

seventy_five_data

survey_minimal_missing= survey[list(seventy_five_data.columnname)]

survey_minimal_missing.reset_index(inplace=True)
X = survey_minimal_missing.drop(columns = 'JobSatisfaction',axis =1)
y = survey_minimal_missing['JobSatisfaction']
# X

In [15]:
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=100, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=100, stratify = y)

In [16]:
pickle.dump(X_test,open('raw_X_test','wb'))
pickle.dump(y_test,open('raw_y_test','wb'))

In [27]:
# X_cat_train = X_train.select_dtypes(include='object')


# list(X_cat_train.columns)[30]

'WorkDatasetSize'

In [29]:
pickle.dump (list(X_cat_train.columns),open('cat_columns.p','wb'))

In [18]:
X_cat_train = X_train.select_dtypes(include='object')
X_cat_train = pd.DataFrame(X_cat_train,dtype ='str')
imp_freq_train = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_cat_imputed_train = imp_freq_train.fit_transform(X_cat_train)
X_cat_imputed_train = pd.DataFrame(data = X_cat_imputed_train)

OE = OneHotEncoder(handle_unknown='ignore')
OE.fit(X_cat_imputed_train)
X_cat_imputed_train = OE.transform(X_cat_imputed_train)
names= OE.get_feature_names()[:]
names = names.tolist()
X_cat_imputed_train = pd.DataFrame(data = X_cat_imputed_train.toarray(), columns = names)




X_num_train = X_train.select_dtypes(include='float64')
imp_freq_train = SimpleImputer(missing_values=np.nan, strategy='median')
X_num_imputed_train = imp_freq_train.fit_transform(X_num_train)
X_num_imputed_train = pd.DataFrame(data= X_num_imputed_train, columns = X_num_train.columns)

scale = StandardScaler()
scale.fit(X_num_imputed_train)
X_num_imputed_train = scale.transform(X_num_imputed_train)
X_num_imputed_train = pd.DataFrame(data= X_num_imputed_train, columns = X_num_train.columns)


X_train_processed = pd.concat([X_cat_imputed_train,X_num_imputed_train],axis =1)

y_train =pd.Series(y_train, dtype ='int')

In [9]:
X_cat_test = X_test.select_dtypes(include='object')
X_cat_test = pd.DataFrame(X_cat_test,dtype ='str')
imp_freq_test = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_cat_imputed_test = imp_freq_test.fit_transform(X_cat_test)
X_cat_imputed_test = pd.DataFrame(data = X_cat_imputed_test)

X_cat_imputed_test = OE.transform(X_cat_imputed_test)
names= OE.get_feature_names()[:]
names = names.tolist()
X_cat_imputed_test = pd.DataFrame(data = X_cat_imputed_test.toarray(), columns = names)




X_num_test = X_test.select_dtypes(include='float64')
imp_freq_test = SimpleImputer(missing_values=np.nan, strategy='median')
X_num_imputed_test = imp_freq_test.fit_transform(X_num_test)
X_num_imputed_test = pd.DataFrame(data= X_num_imputed_test, columns = X_num_test.columns)

scale = StandardScaler()
scale.fit(X_num_imputed_test)
X_num_imputed_test = scale.transform(X_num_imputed_test)
X_num_imputed_test = pd.DataFrame(data= X_num_imputed_test, columns = X_num_test.columns)


X_test_processed = pd.concat([X_cat_imputed_test,X_num_imputed_test],axis =1)

y_test =pd.Series(y_test, dtype ='int')

In [10]:
X_cat_val = X_val.select_dtypes(include='object')
X_cat_val = pd.DataFrame(X_cat_val,dtype ='str')
imp_freq_val = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_cat_imputed_val = imp_freq_val.fit_transform(X_cat_val)
X_cat_imputed_val = pd.DataFrame(data = X_cat_imputed_val)


X_cat_imputed_val = OE.transform(X_cat_imputed_val)
names= OE.get_feature_names()[:]
names = names.tolist()
X_cat_imputed_val = pd.DataFrame(data = X_cat_imputed_val.toarray(), columns = names)




X_num_val = X_val.select_dtypes(include='float64')
imp_freq_val = SimpleImputer(missing_values=np.nan, strategy='median')
X_num_imputed_val = imp_freq_val.fit_transform(X_num_val)
X_num_imputed_val = pd.DataFrame(data= X_num_imputed_val, columns = X_num_val.columns)

scale = StandardScaler()
scale.fit(X_num_imputed_val)
X_num_imputed_val = scale.transform(X_num_imputed_val)
X_num_imputed_val = pd.DataFrame(data= X_num_imputed_val, columns = X_num_val.columns)


X_val_processed = pd.concat([X_cat_imputed_val,X_num_imputed_val],axis =1)

y_val =pd.Series(y_val, dtype ='int')

In [14]:
pickle.dump(X_num_val.columns,open('flask_adjustment_names.p','wb'))
pickle.dump(scale,open('number_scaler.p','wb'))

In [ ]:
# X_imputed.sample(20)

In [ ]:
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.over_sampling import SMOTE, ADASYN


# ros = RandomOverSampler()
# X_train_os, y_train_os = ros.fit_sample(X_train_processed, y_train)


# smot = SMOTE()
# X_train_smote, y_train_smote = smot.fit_sample(X_train_processed,y_train)

# adasyn = ADASYN()
# X_train_ada, y_train_ada = adasyn.fit_sample(X_train_processed,y_train)



In [13]:
pickle.dump(X_train_processed,open('X_train.p', 'wb'))


pickle.dump(y_train,open('y_train.p', 'wb'))

pickle.dump(X_val_processed,open('X_val_processed.p', 'wb'))
pickle.dump(X_test_processed,open('X_test_processed.p', 'wb'))
pickle.dump(y_val,open('y_val.p', 'wb'))
pickle.dump(y_test,open('y_test.p', 'wb'))


## Pickle Load

In [14]:
X_train = pickle.load(open('X_train.p', 'rb'))
y_train = pickle.load(open('y_train.p', 'rb'))

X_val= pickle.load(open('X_val_processed.p','rb'))
y_val= pickle.load(open('y_val.p','rb'))

X_test= pickle.load(open('X_test_processed.p','rb'))
y_test= pickle.load(open('y_test.p','rb'))

# Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegressionCV
logRegModel = LogisticRegressionCV(cv =5, Cs=10,scoring = 'precision')
logRegModel.fit(X_train,y_train)

/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:7

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='precision',
           solver='lbfgs', tol=0.0001, verbose=0)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# import sklearn.metrics as metrics

# # Run CV with 5 folds (Random Forest)

# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [300,400,500],
#     'max_features': ['sqrt'],
#     'min_samples_leaf': [1],
#     'min_samples_split': [2],
#     'n_estimators': [300,500,600]
# }

# rf = RandomForestClassifier()
# rf_grid_os= GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=10, n_jobs=-1, scoring ='roc_auc')
# rf_grid_os.fit(X_train_os, y_train_os)

In [ ]:
# rf_grid.best_score_

In [ ]:
# rf_grid.best_params_

In [ ]:
logRegModelCoef['abs_coef'] = logRegModelCoef['coef'].abs()
logRegModelCoef.sort_values(by = 'abs_coef', ascending = False)

In [ ]:
# parameter_grid={
#     "loss":["deviance"],
#     "learning_rate": [0.6],
#     'min_samples_leaf': [1, 2],
#     'min_samples_split': [2, 3],
#     "max_depth":[40,100,160],
#     "max_features":["sqrt"],
#     "criterion": ["friedman_mse"],
#     "subsample":[ 1.0],
#     "n_estimators":[125,300,500]
#     }
# gb_grid = GridSearchCV(GradientBoostingClassifier(), parameter_grid, cv=5, n_jobs=-1,scoring = 'roc_auc', verbose= True)

# gb_grid.fit(X_train, y_train)

# gb_grid.best_score_


In [ ]:
# parameter_grid={
#     "loss":["deviance"],
#     "learning_rate": [0.6],
#     'min_samples_leaf': [1, 2],
#     'min_samples_split': [2, 3],
#     "max_depth":[40,50],
#     "max_features":["sqrt"],
#     "criterion": ["friedman_mse"],
#     "subsample":[ 1.0],
#     "n_estimators":[125]
#     }
# gb_grid_ada = GridSearchCV(GradientBoostingClassifier(), parameter_grid, cv=5, n_jobs=-1,scoring = 'roc_auc', verbose= True)

# gb_grid_ada.fit(X_train_ada, y_train_ada)

# gb_grid_ada.best_score_

In [ ]:
# parameter_grid={
#     "loss":["deviance"],
#     "learning_rate": [0.6],
#     'min_samples_leaf': [1, 2],
#     'min_samples_split': [2, 3],
#     "max_depth":[40,50],
#     "max_features":["sqrt"],
#     "criterion": ["friedman_mse"],
#     "subsample":[ 1.0],
#     "n_estimators":[125]
#     }
# gb_grid_os = GridSearchCV(GradientBoostingClassifier(), parameter_grid, cv=5, n_jobs=-1,scoring = 'roc_auc', verbose= True)

# gb_grid_os.fit(X_train_os, y_train_os)

# gb_grid_os.best_score_

In [ ]:
# gb_grid_os.predict(X_test_processed)

## Pickle the models

In [ ]:
# gb_grid.best_params_

In [16]:
gb_final = GradientBoostingClassifier(criterion='friedman_mse', learning_rate= .6, loss = 'deviance', 
                                         max_depth=40,min_samples_leaf=1, min_samples_split=3, n_estimators=125,
                                        subsample =1.0)
gb_final.fit(X_train,y_train)

pickle.dump(gb_final,open('gb_final.p','wb'))

In [17]:
# rf_grid_os.best_params_

In [21]:
rf_final = RandomForestClassifier(bootstrap=True, max_depth=100,max_features='sqrt',min_samples_leaf=2,
                                    min_samples_split=2, n_estimators = 300)

rf_final.fit(X_train,y_train)

pickle.dump(rf_final,open('rf_final.p','wb'))

In [22]:
pickle.dump(logRegModel,open('logRegModel.p','wb'))

In [23]:
logRegModel = pickle.load(open('logRegModel.p','rb'))

In [24]:
rf = pickle.load(open('rf_final.p','rb'))
gb = pickle.load(open('gb_final.p','rb'))

In [25]:
X_train = pickle.load(open('X_train.p','rb'))
y_train = pickle.load(open('y_train.p','rb'))

In [26]:
model_names = ["rf","gb","logRegModel"]


model_vars = [eval(n) for n in model_names]
model_list = list(zip(model_names, model_vars))

In [27]:
voting_classifer_soft= VotingClassifier(estimators=model_list,
                                    voting='soft', #<-- sklearn calls this soft voting
                                    n_jobs=-1)
voting_classifer_soft.fit(X_train, y_train)


VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_wei...dom_state=None, refit=True, scoring='precision',
           solver='lbfgs', tol=0.0001, verbose=0))],
         flatten_transform=None, n_jobs=-1, voting='soft', weights=None)

In [28]:
pickle.dump(voting_classifer_soft, open('voting_classifer_soft.p','wb'))